In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('../input/uci-secom.csv')
print(data.shape)
print(data.head(5))
import os

(1567, 592)
                  Time        0        1    ...         588       589  Pass/Fail
0  2008-07-19 11:55:00  3030.93  2564.00    ...         NaN       NaN         -1
1  2008-07-19 12:32:00  3095.78  2465.14    ...      0.0060  208.2045         -1
2  2008-07-19 13:17:00  2932.61  2559.94    ...      0.0148   82.8602          1
3  2008-07-19 14:43:00  2988.72  2479.90    ...      0.0044   73.8432         -1
4  2008-07-19 15:22:00  3032.24  2502.87    ...      0.0044   73.8432         -1

[5 rows x 592 columns]


In [2]:
data.isnull().any().any()

True

In [3]:
d = data.isnull().sum()
j = []
for i in d.keys():
    if(d[i] >900):
        print(i, d[i])
        j.append(i)

85 1341
109 1018
110 1018
111 1018
157 1429
158 1429
220 1341
244 1018
245 1018
246 1018
292 1429
293 1429
358 1341
382 1018
383 1018
384 1018
492 1341
516 1018
517 1018
518 1018
578 949
579 949
580 949
581 949


In [4]:
data.drop(j, axis = 1, inplace = True)

In [5]:
data.replace(np.nan, 0, inplace = True)
#from sklearn.preprocessing import Imputer
#imputer = Imputer(missing_values = np.nan, strategy = 'mean')
data.isnull().any().any()

False

In [6]:
from sklearn.preprocessing import Imputer 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, average_precision_score


In [7]:
data['Pass/Fail'].value_counts()#-1 is pass and 1 is fail

-1    1463
 1     104
Name: Pass/Fail, dtype: int64

In [8]:
x = data.iloc[:, 1:-1]
y = data.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 0)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
print(x_train.shape)
print(x_test.shape)
print(data.shape)

(1096, 566)
(471, 566)
(1567, 568)


In [10]:
(data['Pass/Fail'] == 1).index


RangeIndex(start=0, stop=1567, step=1)

In [11]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

In [12]:
lr = LogisticRegression(C = 1000, penalty = 'l2')
lr.fit(x_train_std, y_train)
y_pred_lr = lr.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_lr, y_test))
cm = confusion_matrix(y_pred_lr, y_test)
print('precision_score', average_precision_score(y_pred_lr, y_test))
print('recall_score', recall_score(y_pred_lr, y_test))
print('f1_score',f1_score(y_pred_lr,y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy score 0.8428874734607219
precision_score 0.14716984611169942
recall_score 0.11290322580645161
f1_score 0.1590909090909091


In [13]:
y_test[y_test == -1].sum()

-445

In [14]:
isolation = IsolationForest()
isolation.fit(x_train_std, y_train)
y_pred_iso = isolation.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_iso, y_test))
cmiso = confusion_matrix(y_pred_iso, y_test)
print('precision_score', average_precision_score(y_pred_iso, y_test))
print('recall_score', recall_score(y_pred_iso, y_test))
print('f1_score',f1_score(y_pred_iso,y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


accuracy score 0.14012738853503184
precision_score 0.895388043917655
recall_score 0.05200945626477541
f1_score 0.0979955456570156


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [15]:
cmiso

array([[ 44,   4],
       [401,  22]])

In [16]:
print(cmiso[1][1]/ (cmiso[1][1] + cmiso[0][1]))

0.8461538461538461


In [17]:
forest = RandomForestClassifier()
forest.fit(x_train_std, y_train)
y_pred_rf = forest.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_rf, y_test))
print('confusion_matrix', confusion_matrix(y_pred_rf, y_test))
print('precision_score', average_precision_score(y_pred_rf, y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy score 0.9426751592356688
confusion_matrix [[444  26]
 [  1   0]]
precision_score 0.0021231422505307855


In [18]:
y_test[y_test == 1].sum()

26

In [19]:
xgb1 = xgb.XGBClassifier(objective = 'binary:logistic', booster = 'gblinear')
xgb1.fit(x_train_std, y_train)
y_pred_xgb = xgb1.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_xgb, y_test))
print('confusion_matrix', confusion_matrix(y_pred_xgb, y_test))
print('precision_score', average_precision_score(y_pred_xgb, y_test))


accuracy score 0.9426751592356688
confusion_matrix [[444  26]
 [  1   0]]
precision_score 0.0021231422505307855


In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C':[5, 10, 250, 100, 500],
    'penalty':['l1', 'l2']
}
gslr = GridSearchCV(estimator = lr, param_grid = param_grid, n_jobs = -1, cv = 10)
gslr.fit(x_train_std, y_train)
y_pred_gslr = gslr.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_gslr, y_test))
print('confusion_matrix', confusion_matrix(y_pred_gslr, y_test))
print('precision_score', average_precision_score(y_pred_gslr, y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy score 0.8662420382165605
confusion_matrix [[403  21]
 [ 42   5]]
precision_score 0.1096302396614092


In [21]:
print('confusion_matrix', confusion_matrix(y_pred_gslr, y_test))

confusion_matrix [[403  21]
 [ 42   5]]


In [22]:
parameters = { 'max_features':np.arange(5,10),'n_estimators':[500],'min_samples_leaf': [10,50,100,200,500]}

random_grid = GridSearchCV(forest, parameters, cv = 5)
random_grid.fit(x_train_std, y_train)
y_pred_gslr = random_grid.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_gslr, y_test))
print('confusion_matrix', confusion_matrix(y_pred_gslr, y_test))
print('precision_score', average_precision_score(y_pred_gslr, y_test))


accuracy score 0.9447983014861996
confusion_matrix [[445  26]
 [  0   0]]
precision_score nan


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train_std, y_train)
y_pred_lda = lda.predict(x_test_std)
print('accuracy score', accuracy_score(y_pred_lda, y_test))
print('confusion_matrix', confusion_matrix(y_pred_lda, y_test))
print('precision_score', average_precision_score(y_pred_lda, y_test))
print('recall_score', recall_score(y_pred_lda, y_test))
print('f1_score',f1_score(y_pred_lda,y_test))

accuracy score 0.8832271762208068
confusion_matrix [[411  21]
 [ 34   5]]
precision_score 0.09684166886518675
recall_score 0.1282051282051282
f1_score 0.15384615384615385


/opt/conda/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
